In [ ]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow.keras as keras
import os

In [ ]:
path = '../datasets/B101/'
files = os.listdir(path)
files = [file for file in files if file.endswith('_trimmed.mat')]

In [ ]:
dataSet = {}
for file in files:
    fileName = os.path.basename(file)
    fileName = fileName.split('.')[0]
    fileName = fileName.removeprefix('Channel')
    fileName = fileName.removesuffix('_trimmed')
    dataSet[fileName] = sio.loadmat(path + file)

In [ ]:
dataSet['1']['trimmedMatrix'][0]

In [ ]:
#Plot the first channel
plt.figure(figsize=(30,10))
plt.plot(dataSet['1']['trimmedMatrix'][0][:500], 'g')
plt.plot(dataSet['2']['trimmedMatrix'][0][:500], 'r')
plt.plot(dataSet['3']['trimmedMatrix'][0][:500], 'b')
#plt.plot(dataSet['4']['trimmedMatrix'][0][:500], 'c')
#plt.plot(dataSet['39']['trimmedMatrix'][0][:500],'y')
plt.show()

In [ ]:
for delta1 in range(10):
    delta2 = delta1*2
    #calculate correlation between 3 channels
    corr1 = np.corrcoef(dataSet['3']['trimmedMatrix'][0][0:500], dataSet['2']['trimmedMatrix'][0][delta1:(500+delta1)])
    corr2 = np.corrcoef(dataSet['3']['trimmedMatrix'][0][0:500], dataSet['1']['trimmedMatrix'][0][delta2:(500+delta2)])
    corr3 = np.corrcoef(dataSet['2']['trimmedMatrix'][0][delta1:(500+delta1)], dataSet['1']['trimmedMatrix'][0][delta2:(500+delta2)])
    print(f'delta1: {delta1}, delta2: {delta2}, corr1: {corr1[0][1]}, corr2: {corr2[0][1]}, corr3: {corr3[0][1]}')

In [ ]:
delta1 = 4
delta2 = delta1*2
plt.figure(figsize=(30,10))
plt.plot(dataSet['1']['trimmedMatrix'][0][delta2:(500+delta2)], 'g')
plt.plot(dataSet['2']['trimmedMatrix'][0][delta1:(500+delta1)], 'r')
plt.plot(dataSet['3']['trimmedMatrix'][0][0:500], 'b')
plt.show()

In [ ]:
#Convert dataset to a DataFrame
sample_number = list(range(1, len(dataSet['1']['trimmedMatrix'][0]) +1 ))
len(sample_number)

In [ ]:
data = {i: dataSet[str(i)]['trimmedMatrix'][0] for i in range(1, len(dataSet) + 1)}
data['Sample Number'] = sample_number

In [ ]:
df = pd.DataFrame(data)
df.columns = df.columns.astype(str)
df['Sample Number'] = df['Sample Number'].astype(str)
#save to csv
df.to_csv('../datasets/B101.csv', index=False)
df.head()

In [ ]:
#Normalize the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
oldDf = df
df = df.drop(columns=['Sample Number'])
scaler.fit(df)
df_normalized = scaler.transform(df)
df_normalized = pd.DataFrame(df_normalized)
#df_normalized.columns = oldDf.columns[:-1]
#df_normalized['Sample Number'] = oldDf['Sample Number']
df_normalized.head()

In [ ]:
#Split the data into training and testing
train_size = 4500
train = df_normalized.iloc[:train_size, :]
test = df_normalized.iloc[train_size:, :]
train.head()

In [ ]:
test.head()

In [ ]:
#Multivariate time series forecasting with LSTMs
model = keras.models.Sequential()
model.add(keras.layers.LSTM(128, activation='relu', return_sequences=True))
model.add(keras.layers.LSTM(128, activation='relu'))
model.add(keras.layers.Dense(62))
model.compile(optimizer='adam', loss='mse')

In [ ]:
#Prepare the data
def prepare_data(data, n_steps):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps
        if end_ix > len(data) - 1:
            break
        seq_x, seq_y = data[i:end_ix, :], data[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
n_steps = 10
X_train, y_train = prepare_data(train.values, n_steps)
X_test, y_test = prepare_data(test.values, n_steps)

In [ ]:
#Ogni finestra è di 1 n_steps campioni. Ci sono train_size-n_steps finestre. Ogni finestra ha n_steps snapshot di input e ogni input è fatto da 62 canali.
X_train.shape, y_train.shape

In [ ]:
#Train the model
model.fit(X_train, y_train, epochs=100, verbose=0)

In [ ]:
#Evaluate the model
model.evaluate(X_test, y_test)

In [ ]:
#Make predictions
predictions = model.predict(X_test)

In [ ]:
X_test.shape

In [ ]:
y_test.shape

In [ ]:
predictions.shape

In [ ]:
#Plot the predictions
plt.figure(figsize=(30,10))
plt.plot(predictions[:,0], 'g')
plt.plot(y_test[:,0], 'r')
plt.show()

In [44]:
#Plot predictions with the original data
channel1Data = X_train[:,0,0]
channel1DataTest = X_test[:,0,0]
plt.figure(figsize=(30,10))


AttributeError: 'numpy.ndarray' object has no attribute 'append'